In [ ]:
import os
os.chdir("../../../..")
import jax
jax.devices()

In [ ]:
from scripts.nj.utils import *
import matplotlib.pyplot as plt

In [ ]:
def generate_connections(
    groups,
    group_complexity,
    group_density=0.5,
    noise=1,
    noise_density=0.3,
    group_recurent_rate=0.5,
):
    return np.repeat(
        np.repeat(
            (1 - group_recurent_rate)
            * (np.random.sample([groups, groups]) < group_density)
            + group_recurent_rate * np.eye(groups, groups),
            group_complexity,
            0,
        ),
        group_complexity,
        1,
    ) * np.random.uniform(
        -1, 1, [group_complexity * groups] * 2
    ) + noise * np.random.uniform(-1, 1, [group_complexity * groups] * 2) * (
        np.random.sample([group_complexity * groups] * 2) < noise_density
    )  # skip connections

In [ ]:

def generate_multiple_connections(C):
    N = C.shape[0]
    return (np.random.uniform(-1, 1, [N] * 3)) * C

g, gc = 10, 1
W = generate_connections(
    g, gc, group_density=1.0, noise=0.01, noise_density=0.0, group_recurent_rate=1.0
)
N = g*gc
W = generate_multiple_connections(W)
W = (W - W.T)

In [ ]:
plt.imshow(W[0])
plt.show()

In [ ]:
def get_my_pipeline(W, dt = 0.1):
    @jax.jit
    def state_transformed(state):
        s, ds = to_diff(state)
        ds['x'] = (s['x'] @ W @ s['x']) * 0.1
        t = s['time']
        ds['x'] = ds['x'].at[0].add(0.2*(jnp.cos(t*np.pi*2 / 100) - ds['x'][0])* (t > 100.0)* (t < 500.0))
        return s, ds

    return get_runge_kutta_step(state_transformed, dt)

In [ ]:
my_pipeline = get_my_pipeline(W, 0.01)

In [ ]:
@jax.jit
def output_transform(state):
    return state['time'], state['x']

initials = {'x':np.random.randn(N), 'time':0.0}

jsim = simulation(initials, my_pipeline, 100, output_transform)
H = jax.block_until_ready(jsim.run(40))

In [ ]:
t, x = H
iS = 3600
plt.plot(t[iS:], x[iS:, 0])
plt.show()